In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_data = pd.read_csv('../data/archive/cross-val/mouth/mouthLabels.csv')

In [3]:
# cropped mouth
train_data = pd.read_csv('../data/archive/cross-val/mouth-cropped/mouthLabels.csv')

In [4]:
Y = train_data[['label']]

In [5]:
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

In [6]:
idg  = ImageDataGenerator(rescale=1./255)

In [16]:
#model = keras.models.load_model("base_yawning_model.h5")
#model = keras.models.load_model("yawning_model.h5")
#model = keras.models.load_model("mobilenet_yawning_model.h5")
#model = keras.models.load_model("base_cropped_yawning_model.h5")
#model = keras.models.load_model("inception_cropped_yawning_model.h5")
model = keras.models.load_model("mobileV2_cropped_yawning_model.h5")

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


In [18]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

for train_index, val_index in skf.split(np.zeros(len(Y)),Y):
	validation_data = train_data.iloc[val_index]
	
	# cropped mouth is (80, 80) else (224, 224) 
	valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = '../data/archive/', target_size=(80, 80),
							x_col = "filename", y_col = "label",
							class_mode = "binary", shuffle = True)
	
	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))
	
	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_LOSS.append(results['loss'])

Found 225 validated image filenames belonging to 2 classes.
8/8 [==============================] - 2s 44ms/step - loss: 1.2218 - accuracy: 0.9022
Found 225 validated image filenames belonging to 2 classes.
8/8 [==============================] - 0s 22ms/step - loss: 2.1033 - accuracy: 0.8489
Found 224 validated image filenames belonging to 2 classes.
7/7 [==============================] - 0s 26ms/step - loss: 1.8619 - accuracy: 0.8616
Found 224 validated image filenames belonging to 2 classes.
7/7 [==============================] - 0s 23ms/step - loss: 1.2959 - accuracy: 0.8884
Found 224 validated image filenames belonging to 2 classes.
7/7 [==============================] - 0s 21ms/step - loss: 1.9399 - accuracy: 0.8482


In [11]:
np.mean(VALIDATION_ACCURACY) # base cropped yawning_model

0.9679285764694214

In [15]:
np.mean(VALIDATION_ACCURACY) # inception cropped yawning_model

0.9964325547218322

In [19]:
np.mean(VALIDATION_ACCURACY) # mobile cropped yawning_model

0.869865071773529

In [10]:
np.mean(VALIDATION_ACCURACY) # base yawning_model

0.8356520533561707

In [9]:
np.mean(VALIDATION_ACCURACY) # inceptionresnetv2 yawning_model

0.9958501338958741

In [12]:
np.mean(VALIDATION_ACCURACY) # mobilenet_yawning_model

0.927481210231781